# <center>Ch 5. PPO微调LLama-3.2 </center>

# 1. PPO原理及应用

```json
    微调没有PPO，就好比吃火锅没有油碟，吃饺子没有醋，汽车没有刹车。
    PPO算法可能不是最显眼的那部分核心，但它是让算法“更加精致”且“口感平衡”的关键成分。就像烹饪中，调味料能让菜肴更加美味，PPO能够让强化学习的过程更加稳定且高效。
    也像汽车的刹车系统，提供了“安全控制”。确保每次策略更新时不会过于激进（就像刹车系统避免车速过快或急转弯导致失控）。如果没有刹车系统，汽车就会变得不稳定，可能会翻车。
```

## PPO是什么？

```json
    PPO（Proximal Policy Optimization近端策略优化）是一种在强化学习（Reinforcement Learning, RL）领域广泛使用的策略梯度算法，常用于训练智能体（比如机器人、游戏AI）在复杂环境中做出决策。它的目标是让模型学会通过与环境的互动逐步优化自己的策略，从而做出更好的决策。
    传统大模型刚出生的时候，并不知道人们更喜欢什么，只能机械的生成对某一部分理解的答案，微调后可能对某些领域会有一些了解，但是在对话输出时只能机械的说出当前领域内的一些知识，但是不能更好的与人进行交互。
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241202161220869.png" width=100%></div>

```json
    首先，SFT（Supervised Fine-Tuning，监督式微调）和强化学习（RL，Reinforcement Learning）都可以提高模型的表现能力，但是在训练大模型时扮演着不同的角色，对应他们的目标和方式也有所不同。
    SFT微调有一定的局限性，SFT微调通常基于大量标注数据进行训练，主要是想让模型在给定输入时候能够产生一个正确的输出，就比如当时第二节课的时候，我们期望让模型记得自己是由木研研发的木木，而不是让模型回答自己是微软研发的。这个过程就是“有监督”的，意味着每个输入都会有一个明确的目标输出。
但这种会有一些限制：
    1、限制模型的回答，他只会回答在特定训练的数据集上，遇到新的场景表现相对不好。
    2、不能很好的根据当前环境进行改变，不好平衡内容质量与合理性，不能根据环境对回答进行调整，比如：我现在坐在包子铺中，我饿了应该吃些什么？ 最好的回答是吃包子，而不是去买一碗面。
    3、难以应对动态的推理与决策：SFT微调通常是针对静态的标注数据集，模型会学习固定的模式。
    4、模型引导性较弱：人工标注的数据集来引导模型学习，很难覆盖所有的场景与任务目标，尤其是那种任务导向的对话，比如要做一碗面，但是模型只学会了和面，后面的切面还是刀削面，模型无法判断。
    5、很难突破自我：模型容易被数据限制，强耦合的回答当前问题，很难超出提供数据之外的回复能力。

强化学习能做什么呢？互补了sft的短板！
    1、允许模型在与环境的交互中实时调整策略，特别是在需要做出长期决策或复杂推理时，能够通过奖励信号引导模型进行自我优化。例如，强化学习可以帮助模型在长对话中更好地理解上下文和用户意图，做出合适的响应。
    2、引导模型使其向着特定的目标：在完成具体任务时根据预设的奖励信号进行优化，（如提高任务完成率、提升用户满意度等）不断调整行为。
    3、能够让广阔的空间中进行不断尝试，从而发现更优的策略，特别是在需要创新或面对不确定性时，能够产生更符合实际需求的解决方案。
    4、通过与环境的互动和反馈机制帮助模型更好地适应新环境或新任务，从而避免过拟合，提升模型的泛化能力。
    5、模型可以在和环境的交互过程中，通过奖励机制进行自我调整，不断优化输出，避免不当的输出。使得生成内容更加丰富、灵活，尤其在多轮对话、任务导向生成等复杂场景下，表现尤为突出。

SFT能够有效地让模型掌握基础的知识和语言能力，但在面对更复杂、动态的任务时，强化学习提供了更强的适应性和优化能力。尤其是在任务导向对话系统、多轮对话、个性化推荐等应用中，强化学习能够显著提升模型的性能，使其能够在实际应用中更好地满足目标和用户需求。
```

## PPO应该作用在哪里？

```json
    有了上边讲解，我们反推一下，PPO补充了SFT微调后的模型，SFT模型又是在base模型基础上微调得来的，所以一个成熟的模型训练顺序粗略的概括为: 预训练得出的base模型——》经过专业引导出来的SFT模型——》使用强化学习补充后的模型——》推出市场
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241202165502496.png" width=100%></div>

```json
这就好比一个是一个大学生刚毕业，然后进入到一家企业开始实习，学习了基础专业知识后。开始使用各种方式来测试自己学到的技能，来方便更好的让自己进步，尽快达到能够独立成为王者的阶段。
前边几节课已经通过使用lora的方式让模型如何学习某一个领域的知识。
接下来就需要让模型反复训练自己能够在多种场景进行更好的回答(现在大模型强化学习阶段出现的算法组合方案层出不穷本次只简单列举三种)：
    方案一：使用PPO+reward(奖励模型)组合成RLHF模式：就好比你这在学打篮球，不断练习投篮和过人技巧，并反复观看自己的练习视频每次进步一点点，并找教练指导、裁判来评分，边学边练。
    方案二：使用DPO+reward(奖励模型)在网上下载一堆视频，不断学习别人好的技巧与坏的技巧，然后进行评判后学习。
    方案三：仅仅使用DPO在网上下载一堆视频，不断学习别人好的技巧与坏的技巧，自我进行练习评判。

本节课涉及的主要为PPO相关概念与算法，主讲方案一的应用，其他方案会在后续课程中进行讲解！
```

```json
使用PPO方法进行微调的模型：
    1. ChatGPT
        优化生成文本的质量和符合人类偏好的能力。通过结合人类反馈，PPO帮助ChatGPT在对话生成中实现更高的自然度和相关性。
    2. InstructGPT
        PPO算法进行微调，通过奖励模型来优化生成的文本，使其更符合用户的期望。
    3. FLAN-T5
        减少生成内容的毒性方面。通过结合Meta AI的仇恨言论奖励模型，帮助FLAN-T5生成更安全和符合道德标准的文本。
    4. GPT-4
        PPO的引入使得GPT-4能够更好地处理用户反馈，从而优化其生成内容的质量。
    5. LLaMA
        结合奖励模型来优化生成文本的质量和相关性。
```

# 2. PPO架构讲解

PPO（Proximal Policy Optimization）的核心目标是通过优化目标函数来最大化累积奖励，同时保证训练过程的稳定性。PPO算法的公式包含了**策略比率**和**剪切机制**，它通过控制策略更新的幅度，避免过度更新导致的训练不稳定。


## 2.1 PPO的公式

PPO的目标函数可以表示为：

$$
L^{CLIP}(\theta) = \mathbb{E}_t \left[ \min \left( r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t \right) \right]
$$

这里，包含了几个重要的组成部分，我们逐个解析。

#### 1. **策略比率** $ r_t(\theta) $

$$
r_t(\theta) = \frac{\pi_\theta(a_t | s_t)}{\pi_{\theta_{\text{old}}}(a_t | s_t)}
$$

- $ r_t(\theta) $ 是 **策略比率**，它表示当前策略 $ \pi_\theta $ 与旧策略 $ \pi_{\theta_{\text{old}}} $ 对于某个动作 $ a_t $ 在状态 $ s_t $ 下的概率比率。
- 也就是说，$ r_t(\theta) $ 衡量了当前策略在给定状态下选择某个动作的概率与旧策略选择该动作的概率之间的比率。
  - 如果 $ r_t(\theta) > 1 $，说明新策略相较于旧策略更加倾向于选择这个动作。
  - 如果 $ r_t(\theta) < 1 $，说明新策略选择这个动作的概率较低。
- **目的**：通过调整 $ r_t(\theta) $ 来最大化优势函数 $ \hat{A}_t $，即选取能够提升长期回报的动作。

#### 2. **优势函数** $ \hat{A}_t $

$$
\hat{A}_t = \text{estimated advantage function at time step 时间步长的估计优势函数 } t
$$

- $ \hat{A}_t $ 是 **优势函数**，它衡量了在某一状态 $ s_t $ 下，选择动作 $ a_t $ 带来的实际回报与期望回报的差异。通常，优势函数是基于 **TD误差**（Temporal Difference Error）或 **GAE（Generalized Advantage Estimation）** 来估算的。
- 具体来说，优势函数可以定义为：
  $$
  \hat{A}_t = Q(s_t, a_t) - V(s_t)
  $$
  其中，$ Q(s_t, a_t) $ 是 **动作价值函数**，表示在状态 $ s_t $ 下选择动作 $ a_t $ 的期望回报；$ V(s_t) $ 是 **状态价值函数**，表示在状态 $ s_t $ 下的期望回报。
- **目的**：优势函数的作用是引导策略更新的方向。正的优势函数值意味着当前的动作比平均策略更好，负的优势函数值则意味着当前的动作不理想。

#### 3. **剪切函数** $ \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) $

$$
\text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon)
$$

- 这是 **剪切（clip）函数**，它确保策略比率 $ r_t(\theta) $ 不会发生过大的变化。
- **剪切机制**的核心思想是：如果 $ r_t(\theta) $ 超出了 $ [1-\epsilon, 1+\epsilon] $ 的范围，则将其“剪切”回这个区间。这意味着，PPO控制策略更新的幅度，不允许策略发生剧烈变化。
  - 如果 $ r_t(\theta) $ 落在 $ [1-\epsilon, 1+\epsilon] $ 范围内，则不做任何修改，保持原有的比率。
  - 如果 $ r_t(\theta) $ 超出了这个区间，剪切函数将其限制在最大值 $ 1+\epsilon $ 或最小值 $ 1-\epsilon $ 上。
- **目的**：通过限制 $ r_t(\theta) $ 的变化幅度，PPO确保每次更新不会偏离原始策略太远，从而避免了过大的策略更新导致训练不稳定。

#### 4. **期望值操作** $ \mathbb{E}_t $

$$
\mathbb{E}_t
$$

- $ \mathbb{E}_t $ 是对时间步 $ t $ 的期望值操作，即对所有时间步 $ t $ 的样本进行平均。
- 这是强化学习中常见的期望操作，表示在多个训练样本上平均损失，确保策略在整个训练过程中能得到较为稳定的优化。

#### 5. **最终的目标函数**

$$
L^{CLIP}(\theta) = \mathbb{E}_t \left[ \min \left( r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t \right) \right]
$$

- 这是PPO的**目标函数**，它由两个部分组成：
  - $ r_t(\theta) \hat{A}_t $ 部分：直接使用当前策略比率和优势函数来优化策略。
  - $ \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t $ 部分：通过剪切机制限制策略比率的变化，从而防止过大的更新。
- **优化过程**：PPO的优化目标是最大化这个目标函数，具体地，选择一个使得目标函数值最大的策略参数 $ \theta $，以便模型能在环境中获得最大的长期回报。

#### 6. **最小化目标函数**

PPO算法的优化过程是通过**最小化损失函数**来实现的，损失函数是目标函数的负值。

$$
\mathcal{L}(\theta) = - \mathbb{E}_t \left[ \min \left( r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t \right) \right]
$$

这个损失函数表示了模型对策略进行调整的方式。通过反向传播最小化损失，模型会更新策略参数 $ \theta $，使得新的策略在环境中的表现更好。


想象你在玩一个**寻宝游戏**，目标是让你的角色找到更多的金币。你已经有了一个策略，这个策略是基于之前的游戏经验学习的，告诉你在每个位置该走哪一步。这个策略可以看作是你的“**游戏规则**”，它决定了你每次做出的选择。

#### 场景目标：试图改进游戏策略

- **旧策略**：你发现过去有一次走向右边的路线很好，虽然有时也走错，但整体而言，大多数时候右边是对的。
- **新策略**：你想改进你的策略，考虑一下是否应该每次都走右边，或者试着左边也走一走。

你知道，如果每次改动都非常大，可能会突然偏离原来的好路线，导致迷路。因此，你要保证“改变不宜过大”，不能每次都去尝试极端的路线。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203142051232.png" width=100%></div>

```json
    滑雪者是Agent，目的是在本次地图中获得更多的金币，但是遇到鱼视为失败，横杠为禁止通行。整张图一开始还是往鱼的方向走，说明游戏会失败-100分，禁止通行是无法进入。
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203145003082.png" width=100%></div>

每次环境在t时刻，会有本身的奖励状态 $R_t$ ，以及环境的集合状态 $S_t$ 会产生 $A_t$ 的行为，同时会产生 $R_t+_1$ $S_t+_1$ 的状态累计

目的**找到一个策略，这个策略根据当前观测到的环境状态和奖励反馈，来选择最佳的动作。**


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203142105212.png" width=100%></div>

```json
    每次失败后，或是无法通行，会逐步减少某一方向的输出概率，甚至可能会将为0，这就是Agent学习到的状态。
```

```json
根据上述关键点，发现需要多个模型写作才能完成PPO算法：
    1、不能脱离原模型，就需要有个参考模型
    2、本身自己微调模型
    3、帮助改进的模型，奖励模型 reward
    4、协助预测每一步预测值，critic 模型
```

## 2.2 架构

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203163522690.png" width=100%></div>

这几个模型的关系我们应该怎么去理解呢？
```json
背景与概述：
    假设现在是一场篮球比赛，教练想通过这场篮球比赛尝试对你进行实战训练来让你逐渐进步。
    但是也不想让你进步的太过离谱，比如一跳8米，全场一步投篮。所以带上了，从小和你一起训练水平接近甚至完全一样的双胞胎兄弟，来进行参考，防止你进步的特别离谱。
    通过每次裁判分数，判断是否能够赢得比赛，拿到比赛最高分。
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203165012011.png" width=100%></div>

- **model（你）**：你是训练中的主角，学习打篮球，目标是提高自己的篮球技巧和比赛表现。
- **pre_model（双胞胎兄弟）**：相同能力的双胞胎兄弟，他和你有相似的基础技能。你们俩在同样的训练环境下，能力相差不大。
- **critic model（篮球教练）**：篮球教练负责预测评估你的动作、提供反馈，告诉你哪些动作做得好，哪些做得不好。目标是帮助你提高打球的技巧。
- **reward model（裁判）**：裁判负责评估你的整体表现，给你打分，实际真实告诉你打得好还是不好。裁判不仅给出即时的反馈，还能基于最终比赛结果（例如得分、赢得比赛等）评估你整体的表现。

 **角色技能**

1. **你和双胞胎兄弟（model 与 pre_model）**
   - **你（model）**和你的双胞胎兄弟（pre_model）在同样的篮球场上训练。你们的技术水平非常相近，都知道如何投篮、传球、跑位等基本技能。(sft后的模型)
   - 但是，你的目标是通过不断练习提升自己的技能，所以你在每一场训练后都会尝试调整自己的动作和策略。
   - 在这里，**双胞胎兄弟（pre_model）**代表了之前的策略，他的表现和你在过去的训练中非常相似，作为对比帮助你判断新的策略是否比原来的策略更好。

2. **篮球教练（critic model）**
   - 在你训练的过程中，**篮球教练（critic model）**会观察你和双胞胎兄弟的表现。教练会告诉你每个动作的优缺点，(因为有双胞胎兄弟做对比所以可以知道哪里是优点)指出你可以改进的地方。
   - 教练会给你具体的反馈，告诉你：比如“你投篮姿势不够标准，应该让手臂伸直一点”，或者“你的跑位不够快，要跑到空位去接球”。
   - 教练的任务是帮助你区分哪些动作能带来更多的成功，哪些动作需要改进。他提供的反馈是 **优势函数（Advantage Function）**，帮助你决定接下来应该朝哪个方向努力。

3. **裁判（reward model）**
   - **裁判（reward model）**则是在每一场比赛后给你打分。裁判不仅根据你是否得分来评价你，还会综合考虑你在比赛中的表现，比如进攻、防守、跑位、团队协作等。
   - 你的得分越高，裁判会给你越高的评价。这种评价是 **奖励信号**，意味着你在比赛中表现得更好，策略更加有效。
   - 裁判不仅仅给出即时反馈，也会根据比赛的结果给你一个整体的评分，帮助你了解自己在长远比赛中的表现如何。

**PPO的训练过程：**

1. **第一步：你开始训练**
   - 开始时，你并不完全知道哪种打法最有效(那就自有发挥吧)。你在篮球场上尝试不同的动作，比如投篮、传球、突破等等。你会在每一场训练中不断尝试改进自己的表现，逐渐学习。
   - 你的双胞胎兄弟（pre_model）会和你做相似的操作，同样场景下他还是按照原来的打发，投篮体态、传球、防守。

2. **第二步：篮球教练（critic model）给出反馈**
   - 每次训练后，教练（critic model）会仔细观察你和双胞胎兄弟的动作。教练告诉你，哪些动作做得好，哪些做得不好。比如，教练可能会告诉你：“你投篮的角度不够好，下次应该尝试投得更直一些。刚刚你的双胞胎兄弟就在这个位置上投的更直，效果更好”，对比之下帮助你理解你的动作和策略是否正确。
   - 教练的反馈相当于 **优势函数（Advantage Function）**，告诉你某个动作相对于其他动作的好坏。
   - 教练会根据你的进步调整对预期的评估，比如你在侧面投了一个三分，相对原来进步很大，教练给你奖励按摩十分钟，但是下次你在相同场景相同位置再投了一个三分，教练就只会给你按摩一分钟。教练也会根据你的进步来调整策略。

3. **第三步：裁判（reward model）给出奖励**
   - 在训练的最后，裁判会根据比赛的表现（比如得分和胜利）给你打分。如果你投中了很多球，或者你的团队赢得了比赛，裁判会给你一个高分。如果你失误很多，裁判的评分就会低。
   - 裁判给出的评价就是 **奖励信号**，你综合下来教练指导打完全场能得到多少分，都是完成后得分。

4. **第四步：调整策略**
   - 你将从教练那里得到的反馈和裁判的评分结合起来，来调整自己的策略。如果你发现某个动作或者打法让你得分更多，那么你就会在下次训练中更加频繁地使用这种策略。比如先防守再投篮，命中率极高。
   - 你的双胞胎兄弟（pre_model）也会根据你的优势进行一点策略调整，和你一起进行对比方便你进步的更多一些。

5. **第五步：剪切机制（防止过度改变）**
   - 假设你每次调整策略时，都非常激进（比如，今天决定完全不投篮，而改成了只靠传球，又或者今天状态极好一跳八米）。这种大幅度的改变可能会让你偏离最有效的打法，导致训练不稳定忽高忽低。
   - 在PPO中，**剪切机制**的作用类似于防止你和双胞胎兄弟之间的策略变化过大。假如你调整策略的幅度太大，教练（critic）和裁判（reward model）会提醒你：这种调整可能过于极端，不符合你的长远发展。你应该做出渐进式的小幅度调整，逐步提高自己的技能。

6. **反复训练**
   - 随着时间的推移，你会通过这种方式不断提高自己的篮球技巧。每次训练结束后，你都会得到教练的反馈和裁判的评分，继续调整策略和动作，优化自己的表现。
   - 你和双胞胎兄弟之间的对比会帮助你评估哪些调整是有效的，哪些调整则没有效果。

在这个打篮球的场景中：

   - **你（model）** 是训练中的主体，目标是不断提高自己的篮球技能。
   - **双胞胎兄弟（pre_model）** 是之前的策略，作为对比帮助你判断改进后的策略是否更好。
   - **篮球教练（critic model）** 给出具体的反馈，告诉你哪些动作需要改进，是调整策略的指导者。
   - **裁判（reward model）** 基于你的整体表现（如得分、比赛结果等）给你打分，作为训练的最终目标，告诉你策略的优劣。


## 2.3 微调中的应用

映射到大模型微调中，如何应用呢：
```json
    step1、训练前需要先准备好SFT模型，让他来产生专业领域数据
    step2、准备好reward模型，方便对模型输出做出评分
    Step3、将模型进行融合后训练
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203195258759.png" width=100%></div>

粗略的逻辑是：<br>
1、输入prompt<br>
2、Actor基于当前状态内容开始生成内容$A_t$<br>
3、critic模型基于Actor产生的状态$A_t$开始预测下一步的策略，来研究是否在能让Actor产生长期价值<br>
4、Actor生成完成后，reward模型开始Actor生成内容评分。<br>
5、critic基于reward的评分高低，并对比reference模型内容防止更新过大。<br>
6、Actor开始更新参数，critic同步也会更显参数，来进行下一步预测判断。

这几个模型在流程中的作用：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203220152007.png" width=100%></div>

```json
    给模型一个prompt，让模型能生成符合人类喜好的response。
    每个时刻只产生一个token，即token是一个一个蹦出来的，先有上一个token，再有下一个token。
    就对应每一个生成，结合之前内容之前就相当于是状态更新 也会对应累计R与V
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203220213645.png" width=100%></div>

```json
    我们希望训练出来的Actor模型既能达到符合人类喜好的目的，又尽量让它和SFT模型不要差异太大,选用KL散值进行计算。
    ref_log_probs - log_probs 差值越小说明两个模型输出的内容越相似，也就代表Actor model更新的步伐不大，就符合预期。
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203220224895.png" width=100%></div>

```json
    reward model： 用来判断即时收益，是真实的得分，并且在训练中是冻结状态，只负责打分。
    生成方式：可以自行基于人类反馈数据(偏好数据)自行训练，也可以引入外部训练好的模型。
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203220235756.png" width=100%></div>

```json
    critic model： 是会随着Actor 更新而变化。
    作用：协助Actor model进行进化，不但在reward拿高分生成符合结果内容，还要帮助Actor猜测人类喜欢生成内容的方向。
    初始化的种类很多：可以使用base model，与Actor一致 或者与reward model一致等。有的人也会称为value model。
```

# 3. 模型微调与验证

## 3.1 模型与数据选取

如何选择模型做为基座模型？

| 特性               | 预训练基座模型 (Pretrained Model)                                                                 | Instruction-tuned 模型 (Instruction-tuned Model)                                               |
|--------------------|---------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------|
| 任务适应性         | 泛化能力强，适应多种自然语言任务                                                                 | 专注于具体任务和指令，更适应任务特定的需求                                                   |
| 微调效率           | 需要较多的训练数据和计算资源，特别是对于任务特定的任务                                          | 在特定任务或指令下可能微调更高效                                                             |
| 适用场景           | 通用 NLP 任务、无标签数据的情况                                                                  | 需要强烈指令理解和人类反馈的任务（如 RLHF）                                                  |
| 生成质量和任务理解 | 对大多数任务有较好性能，但不一定优化任务目标                                                     | 能更好地理解具体任务要求，生成任务相关反馈                                                   |
| 资源消耗           | 需要更多计算资源和训练时间                                                                       | 根据任务优化，通常能较快微调，但依赖于任务数据                                                |

### 总结
- 使用**预训练基座模型**适合任务范围较广、数据多样或不完全标注的情况，且你希望利用预训练模型的丰富语言理解能力。
- 使用**Instruction-tuned模型**更适合于有明确任务目标、需要人类反馈优化或策略评估的场景，尤其是RLHF任务中，能够更精准地理解任务要求并优化奖励信号。

同一个基座模型可以用来训练出reward模型，然后再用这个基座模型进行PPO微调<br>
例如：Qwen2.5-7B-Instruct微调出reward model 为Qwen2.5-7B-rm<br>
可以使用Qwen2.5-7B-Instruct 做PPO微调，然后用Qwen2.5-7B-rm在PPO微调中做奖励判断。<br>

| 方案 | 优点 | 缺点 |
| --- | --- | --- |
| **使用相同基座模型** | - 一致性和兼容性：奖励模型和PPO模型共享相同的表征空间。<br>- 数据共享：能够共享表示学习，提高学习效率。<br>- 训练速度较快：减少需要微调的参数量，节省计算资源。 | - 可能的过拟合：特别是在任务简单或数据不足时，两者可能相互过拟合。<br>- 难以细化奖励模型：共享基座模型可能限制了对奖励模型本身的优化。 |
| **使用不同基座模型** | - 奖励模型优化独立性：奖励模型可以独立优化其参数，不受PPO策略模型影响。<br>- 更强的模型灵活性：允许在奖励建模和策略学习之间有更大的灵活性。<br>- 减少模型共享的偏见：有助于得到更加准确和公平的奖励评估。 | - 计算成本较高：分别微调两个模型会增加计算资源消耗。<br>- 训练复杂度增加：协调两个模型之间的互动变得更加复杂。 |

### 实际应用中的考虑
- **简单任务和小规模模型**：使用相同基座模型简化训练过程。
- **复杂任务和大规模模型**：使用不同的基座模型带来更好的性能。
- **资源和时间有限**：优先选择相同基座模型以提高效率。
- **多任务学习**：不同基座模型为每个任务设计更合适的模型。

### 一些实践建议
- 使用大参数模型做小模型的reward相对更好，容易对小模型产生的内容充分理解并客观评分。
- 最好基座模型与reward模型选择同系列不同参数进行训练，减少出现精度不一致带来的错误
- 也可以使用同系列变种模型作为reward模型：比如开开源llama系列的reward做llama系列模型的reward。

## 3.2 微调及验证

- step1 准备模型

准备基座模型

本次使用Llama-3.2-1B-Instruct https://www.modelscope.cn/models/AI-ModelScope/Llama-3.2-1B-Instruct/files

modelscope download --model AI-ModelScope/Llama-3.2-1B-Instruct --dir ./

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203231253871.png" width=100%></div>

验证模型完整性

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203231652011.png" width=100%></div>

- step2 下载reward模型

```json
    不知道哪个模型比较好的小伙伴可以先去hugging face上看一下榜单进行选取(科学上网)
    https://huggingface.co/spaces/allenai/reward-bench
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241202181656590.png" width=100%></div>

```json
    可以根据自己服务器资源状况选择适当reward模型
    https://huggingface.co/models?sort=downloads&search=reward
    目前hugging face 下载较多的是 Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 有条件的小伙伴可以进行下载
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241202180444235.png" width=100%></div>

```json
    国内小伙伴可以考虑skywork/Skywork-Reward-Models
    modelscope download --model skywork/Skywork-Reward-Models --dir ./
    https://www.modelscope.cn/models/skywork/Skywork-Reward-Models 效果也不错
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241202181601429.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241202181950667.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241202183524350.png" width=100%></div>

校验模型完整性

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203232604460.png" width=100%></div>

Skywork-Reward-Gemma-2-27B 和 Skywork-Reward-Llama-3.1-8B 是基于gemma-2-27b-it 和**Meta-Llama-3.1-8B-Instruct**架构构建的两个高级奖励模型。这两个模型均采用Skywork奖励数据集进行训练，该数据集中仅包含从公开数据源获取的8万对高质量偏好对。

与基座模型属于同系列模型

- step 3 准备数据

```json
数据集质量：不必要求所有回答都是完美的，但应尽量确保数据集包含一些优质、标准化的回答，同时也要适当包含一些普通或劣质的回答，以帮助模型学习如何生成多样化和适应性的输出。
    数据一定要均匀分布，包含完美回答，一般回答、劣质回答最好。如果都是完美回答模型就没有改进算法的办法了，都是劣质回答会导致模型不知道什么是完美回答，难以收缩。
    如果只有完美回答或者劣质回答，会导致模型在训练时候会过度倾向好的回答进行训练，最终优化某些输出，从而忽视了其他部分潜在良好的回答。

```

本次选择llama_factory自带数据集 https://github.com/hiyouga/LLaMA-Factory/blob/main/README_zh.md#%E6%95%B0%E6%8D%AE%E9%9B%86

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241203235137862.png" width=100%></div>

 https://github.com/tatsu-lab/stanford_alpaca/blob/main/alpaca_data.json

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241128210359637.png" width=100%></div>

重命名 alpaca_data_llm_ppo.json 进行预览

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241128210502356.png" width=100%></div>

```json 
选取前1000 条 
import json
import sys

def read_first_n_records(input_file_path, output_file_path, n=1000):
    # 读取原始 JSON 文件
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # 获取前 n 条记录
    first_n_records = data[:n]
    
    # 将前 n 条记录保存到新的 JSON 文件
    with open(output_file_path, 'w', encoding='utf-8') as file:
        json.dump(first_n_records, file, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: python script.py <input_file_path> <output_file_path>")
        sys.exit(1)

    input_file_path = sys.argv[1]
    output_file_path = sys.argv[2]

    read_first_n_records(input_file_path, output_file_path)
```

当然，为了数据质量更高可以去hugging face中进行下载偏好数据集进行下载选取高分答案  https://huggingface.co/datasets?sort=trending&search=reward

选择 Skywork/Skywork-Reward-Preference-80K-v0.2

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241127184018498.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241127184308231.png" width=100%></div>

选择过滤器进行过滤

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241127202952941.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241127203018556.png" width=100%></div>

上传服务器后对数据进行预览：
import pandas as pd # 使用 pandas 和 pyarrow 读取 Parquet 文件 df = pd.read_parquet('your_file.parqute') # 打印前20行 print(df.head(20))

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241127201009977.png" width=100%></div>

```json
# 感觉看的不完整可以使用下面的代码
# 设置 Pandas 输出的列宽限制

pd.set_option('display.max_colwidth', None)

# 打印 DataFrame 中的前 20 行

print(df.head(20))

如果想看第几14行

print(df['chosen'].iloc[13])  # 查看第 14 行 'chosen' 列的数据
print(df['rejected'].iloc[13])  # 查看第 14 行 'rejected' 列的数据
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241127210834746.png" width=100%></div>

```json
 选择一个source类型进行过滤 wildguard

 import pandas as pd

# 读取 parquet 文件
file_path = '/home/util/muyan/download_data/train-00000-of-00001.parquet'
df = pd.read_parquet(file_path)
# 过滤 source 为 'wildguard' 的数据
filtered_df = df[df['source'] == 'wildguard']
# 函数用于提取内容

def extract_content(row, column):
​    if len(row[column]) > 0:
​        return row[column][0]['content']
​    return ""

# 创建新的 DataFrame 来存储 Alpaca 格式的数据

alpaca_format = []
for _, row in filtered_df.iterrows():
​    instruction = extract_content(row, 'chosen')
​    output = extract_content(row, 'rejected')
​    alpaca_format.append({'instruction': instruction, 'output': output})
# 打印前几条记录以检查结果

print(alpaca_format[:5])
# 将结果保存为 JSON 文件（可选）

with open('alpaca_dataset.json', 'w') as f:
​    json.dump(alpaca_format, f, indent=4, ensure_ascii=False)
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241128104422178.png" width=100%></div>

```json 
#读取上边过滤出的500条数据  转变成100条数据，前500条数据使用优质回答，后500条使用拒绝回答数据
# 或者直接去取前一千条优质数据 转换为alpace格式
import json
import sys

def convert_to_alpaca_format(input_file_path, output_file_path):
    # 读取原始 JSON 文件
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # 获取前 500 条记录
    first_500_records = data[:500]
    
    # 创建新的列表来存储 Alpaca 格式的数据
    alpaca_format = []
    
    # 处理前 500 条记录，使用 chosen 的 value
    for record in first_500_records:
        instruction = record["conversations"][0]["value"]
        input_field = ""  # 假设没有人类输入字段，如果有请根据实际情况修改
        output = record["chosen"]["value"]
        alpaca_format.append({
            "instruction": instruction,
            "input": input_field,
            "output": output
        })
    
    # 处理后 500 条记录，使用 rejected 的 value
    for record in first_500_records:
        instruction = record["conversations"][0]["value"]
        input_field = ""  # 假设没有人类输入字段，如果有请根据实际情况修改
        output = record["rejected"]["value"]
        alpaca_format.append({
            "instruction": instruction,
            "input": input_field,
            "output": output
        })
    
    # 打印前几条记录以检查结果
    print(alpaca_format[:5])
    
    # 将结果保存为 JSON 文件（可选）
    with open(output_file_path, 'w', encoding='utf-8') as file:
        json.dump(alpaca_format, file, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: python script.py <input_file_path> <output_file_path>")
        sys.exit(1)

    input_file_path = sys.argv[1]
    output_file_path = sys.argv[2]

    convert_to_alpaca_format(input_file_path, output_file_path)
```

- step 4 注册数据集

数据集重命名并转移

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204103300144.png" width=100%></div>

注册数据集
```json
"alpaca_data_llm_ppo": {
    "file_name": "alpaca_data_llm_ppo.json"
  },
  "alpaca_data_llm_ppo_1000": {
     "file_name": "alpaca_data_llm_ppo_1000.json"
  },
  "top1000_500chosen_500rejected": {
    "file_name": "top1000_500chosen_500rejected.json"
  },
```
注册了多份数据集，大家可以自行扩展。


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204104028075.png" width=100%></div>

- step 4 界面查看数据注册是否成功，并生成执行命令。

切换conda环境 conda activate llama_factory

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241128114404966.png" width=100%></div>

进入llama_factory路径下 nohup llamafactory-cli webui >20241128.log 2>&1 &

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241128114554502.png" width=100%></div>

打开界面开始生成命令

选择以下关键点，除PPO外与模型路径其余，其余皆可以根据资源自行调配

微调方法选择lora

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204105030030.png" width=100%></div>

配置reward model

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204105512682.png" width=100%></div>

点击预览命令

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204110056396.png" width=100%></div>

复制命令进行更改，直接复制的命令在脚本中无法直接使用
```json
stage: ppo # **stage: ppo** 改变类型
do_train: True
model_name_or_path: /home/util/muyan/model/AI-ModelScope/Llama-3.2-1B-Instruct
preprocessing_num_workers: 16
finetuning_type: lora
template: llama3
flash_attn: auto
dataset_dir: data
dataset: alpaca_data_llm_ppo_1000
cutoff_len: 2048
learning_rate: 5.0e-05 # 将原来的5e-05 改为 5.0e-05否则会报错
num_train_epochs: 3.0
max_samples: 1000 # 根据资源进行配置 数据集*epochs 但是限制最高max_samples 条数，建议减少数据集比限制条数更省资源
per_device_train_batch_size: 1
gradient_accumulation_steps: 8
lr_scheduler_type: cosine
max_grad_norm: 1.0
logging_steps: 5
save_steps: 100
warmup_steps: 0
packing: False
report_to:none
output_dir: saves/Llama-3.2-1B-Instruct/lora/train_2024-12-04-10-47-43
plot_loss: True
ddp_timeout: 180000000
optim: adamw_torch
lora_rank: 8
lora_alpha: 16
lora_dropout: 0
lora_target: all
reward_model: /home/util/muyan/model/skywork/Skywork-Reward-Models
reward_model_type: full # 将lora 改为full，因为这里使用的是全量的reward模型，不能使用lora参数
top_k: 0
top_p: 0.9
```


learning_rate: 5e-05

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241128222206327.png" width=100%></div>

使用全量模型配参数reward_model_type: lora 报错

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241128222056309.png" width=100%></div>

PPO与lora的关系

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204203838502.png" width=100%></div>

lora、freeze、full 都是让模型学习的方式，这个方式在预训练之后的阶段中都可以使用。

- step 5 复制修改后参数创建指定的.yarml文件

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204112331953.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204112616585.png" width=100%></div>

后端执行 nohup llamafactory-cli train config/ppo_0_5_2024_12_02_18_00.yaml > 20241202_18.log 2>&1

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204112736216.png" width=100%></div>

开始监控日志

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204112959530.png" width=100%></div>

tail -f 20241202_18.log

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204113058247.png" width=100%></div>

查看运行时长

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241129125301139.png" width=100%></div>

监控资源 watch -n 1 nvidia-smi

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241128144350187.png" width=100%></div>

完成后下载损失与奖励训练图

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241129141707960.png" width=100%></div>

查看奖励函图与损失函数图。(如果想要查看详细数据，建议接入Tensorboard 详细操作请看《Ch 3. LLama_factory&Lora微调Qwen2.5做NL2sql》)

qwen系列 基座qwen2.5-0.5B reward使用qwen2.5-3B自行微调
```json
    参数不变的情况下使用的是
    图一：使用与reward 完全数据集进行训练 1000条
    图二：使用微调reward模型对应一半好一半坏数据集共计1000条
    图三：使用微调reward模型对应100条好数据集
```

<div align=center>
        <td><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204115654875.png" width=70%></td>
        <td><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204115715276.png" width=70%></td>
        <td><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204115745766.png" width=70%></td>
</div>

llama系列 基座llama3.2-1B reward使用Skywork-Reward-Models
```json
    参数不变的情况下使用的是
    图一：使用与reward 完全数据集进行训练 1000条
    图二：使用微调reward模型对应一半好一半坏数据集共计1000条
    图三：使用微调reward模型对应100条好数据集
```

<div align=center>
        <td><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204115933082.png" width=70%></td>
        <td><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204120003222.png" width=70%></td>
</div>

- step 6 校验

原生模型

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241201210352067.png" width=100%></div>

PPO微调后

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241201210251847.png" width=100%></div>

llama系列

原生与PPO微调后的

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241201210518833.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241201210717973.png" width=100%></div>

## 3.3 PPO中的full与lora

这里的reward_model_type: full # 将lora 改为full，因为这里使用的是全量的reward模型，不能使用lora参数

如果我们都是用qwen2.5-3B作为基座模型训练reward，然后用生成的reward模型来训练PPO，无疑根据PPO原理会加载四遍基座模型，如果是更大参数在微调中极其耗费资源的<br>
但如果你的reward使用lora微调，是不是只加载训练好的lora块和基座模型融合就能作为reward model。同理criticmodel也用相同的lora块就又剩下一份基座模型的空间。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204122357572.png" width=100%></div>

如果reward_model_type: lora 使用全参模型就会报错

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204122720628.png" width=100%></div>

其他报错：非同系列模型，选择计算维度不一致等导致的 bf16:true

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241202194000162.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241202220543901.png" width=100%></div>

# 4. LLama_factory源码讲解

查找命令执行文件 which llamafactory-cli

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204143839942.png" width=100%></div>

查看内容 cat /root/anaconda3/envs/llama_factory/bin/llamafactory-cli

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204143925123.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204144053237.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204144109889.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204144206607.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204144555600.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204144654046.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204144752483.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204144813695.png" width=100%></div>

主要步骤在step方法中

**Step方法**如何通过 **Proximal Policy Optimization (PPO)** 算法进行 **模型训练**，
代码的主要目标是通过 **奖励信号**（来自奖励模型）、**策略损失**（来自 actor model）和 **价值损失**（来自 critic model）来优化模型的 **策略**

主要作用是通过 **PPO 优化算法** 完成以下任务：
1. **数据预处理**：检查和准备输入数据（`queries` 和 `responses`）。
2. **奖励计算**：通过 **奖励模型** 计算每个响应的 **奖励**，然后计算 **KL 散度** 来控制策略更新幅度。
3. **优势函数和回报计算**：计算 **优势函数** 和 **回报**，用于优化 **actor model**。
4. **模型更新**：计算 **策略损失** 和 **价值损失**，通过 **PPO** 更新模型参数。
5. **训练过程控制**：包括 **早停** 和 **优化步骤**。
6. **日志记录和统计**：记录和输出训练过程中的统计信息，包括奖励、KL 散度、损失等。

#### **1. 参数预处理和安全检查 (`_step_safety_checker`)**

```python
queries, responses, scores, response_masks = self._step_safety_checker(
    bs, queries, responses, scores, response_masks
)
```

- **作用**：这一步会对输入的数据进行 **安全检查** 和 **预处理**。比如检查数据的类型是否匹配、是否有缺失值等。
- **`queries`**：输入问题或任务（指令）。
- **`responses`**：模型生成的响应。
- **`scores`**：奖励分数，用来表示模型生成的响应的质量。
- **`response_masks`**（可选）：响应的掩码，用于指示哪些响应部分是有效的。无效地方会被标记为0

#### **2. 奖励信号的归一化和裁剪**

```python
if self.config.use_score_scaling:
    # 进行奖励分数的归一化 类似将qlora中奖值映射到同一个范围内
    scores_mean, scores_std = self.running.update(scores)
    tensor_to_kwargs = dict(dtype=scores.dtype, device=scores.device)
    score_scaling_factor = self.running.std.to(**tensor_to_kwargs) + torch.finfo(scores.dtype).eps
    if self.config.use_score_norm:
        scores = (scores - self.running.mean.to(**tensor_to_kwargs)) / score_scaling_factor
    else:
        scores /= score_scaling_factor

if self.config.score_clip is not None:
    # 对奖励分数进行裁剪 防止奖励值过大或过小导致模型训练过程中的不稳定性
    scores_dtype = scores.dtype
    scores = torch.clip(scores.float(), -self.config.score_clip, self.config.score_clip).to(dtype=scores_dtype)
```

- **作用**：这部分对 **奖励信号**（`scores`）进行 **归一化** 和 **裁剪**，以确保奖励值在一个适当的范围内。这有助于避免奖励值的极端波动，导致训练不稳定。
  - **`use_score_scaling`**：归一化奖励分数，防止奖励值的方差过大。
  - **`score_clip`**：裁剪奖励分数，防止奖励分数过大或过小，从而影响训练过程的稳定性。

#### **3. 比较和推送最好的模型（逻辑上没什么意义）**

```python
if hasattr(self, "highest_reward"):
    if self.compare_step % self.config.compare_steps == 0:
        curr_mean_reward = scores.mean()
        if curr_mean_reward > self.highest_reward:
            self.highest_reward = curr_mean_reward
            # 推送模型到 hub
            self.push_to_hub(**self.push_to_hub_kwargs)
    self.compare_step += 1
```

- **作用**：这部分代码检查当前训练步骤的 **奖励值** 是否为历史最高奖励。如果是，则将模型推送到 **模型仓库**（例如 Hugging Face Hub）。这有助于保存当前训练最优的模型版本。

#### **4. 准备输入数据和处理分布式设置**

```python
model_inputs = self.prepare_model_inputs(queries, responses)
```

- **作用**：这部分准备输入数据，包括 **`queries`** 和 **`responses`**，将它们转换为模型所需的格式。
- 还包括对分布式训练环境的支持，比如 **填充** 输入数据（`pad_across_processes`）以确保在多个设备上的训练能正确运行。

#### **5. 前向传播并获取模型输出（计算 log-probabilities、values 等）**

```python
all_logprobs, logits_or_none, values, masks = self.batched_forward_pass(
    self.model,
    queries,
    responses,
    model_inputs,
    response_masks=response_masks,
    return_logits=full_kl_penalty,
)
```

- **作用**：调用模型的前向传播（`batched_forward_pass`）：
  - **`logprobs`**：计算生成 **`responses`** 的对数概率，用于计算 **策略损失**（policy loss）。
  - **`values`**：计算每个状态的 **价值估计**（`values`），用于计算 **价值损失**（value loss）。
  - **`masks`**：用于指示哪些响应部分有效的掩码。
  - 通过 **`ref_model`**（参考模型）计算 **`ref_logprobs`**，用于计算 **KL 散度**。

#### **6. 计算奖励和 KL 散度**

```python
rewards, non_score_reward, kls = self.compute_rewards(
    scores, active_full_logprobs, ref_full_logprobs, masks
)
```

- **作用**：计算每个生成响应的 **奖励**（`rewards`）：
  - **`compute_rewards`** 会根据 **`scores`**（奖励模型评分）以及 **`logprobs`**（来自当前模型）和 **`ref_logprobs`**（来自参考模型）计算 **奖励**。这一步是 PPO 中的关键，奖励信号引导 **actor model** 更新策略。
  - **KL 散度**（`kls`）是用来衡量当前模型与参考模型之间的差异的。如果 **KL 散度** 过大，会导致策略更新过大，训练不稳定。

#### **7. 计算优势函数和回报**

```python
values, advantages, returns = self.compute_advantages(values, rewards, masks)
```

- **作用**：根据 **奖励**（`rewards`）和 **状态价值**（`values`）计算 **优势函数**（`advantages`）和 **回报**（`returns`）。
  - **优势函数**：表示当前动作相对于当前状态的价值的好坏，帮助 **actor model** 优化策略。
  - **回报**：通过 **`rewards`** 和 **`values`** 计算，表示当前状态下的长期期望回报。

#### **8. 执行 PPO 训练步骤**

```python
train_stats = self.train_minibatch(
    mini_batch_dict["logprobs"],
    mini_batch_dict["values"],
    logprobs,
    logits,
    vpreds,
    mini_batch_dict["masks"],
    mini_batch_dict["advantages"],
    mini_batch_dict["returns"],
)
```

- **作用**：通过 **PPO 更新** 来优化 **actor model**。这部分代码计算 **策略损失**（policy loss）和 **价值损失**（value loss），并通过这些损失更新模型的参数。

#### **9. 早停控制和优化过程的执行**

```python
if self.config.early_stopping:
    policykl = train_stats["policy/policykl"]
    early_stop = self._early_stop(policykl)
    if early_stop:
        break
```

- **作用**：这部分代码实现了 **早停** 机制。当模型的 **KL 散度** 超过一定阈值时，提前停止训练，防止过拟合。

#### **10. 统计信息和日志记录**

```python
stats = self.record_step_stats(
    scores=scores,
    logprobs=all_logprobs,
    ref_logprobs=ref_logprobs,
    non_score_reward=non_score_reward,
    train_stats=train_stats,
    kl_coef=self.kl_ctl.value,
    masks=masks,
    queries=queries,
    responses=responses,
    kls=kls,
)
```

- **作用**：记录当前训练步骤的统计信息，例如 **奖励**（`scores`）、**KL 散度**（`kls`）、**训练统计信息**等。这些信息可以用于调试、分析模型性能，并用于 **TensorBoard** 或其他日志工具进行可视化。

#### **11. 训练结束后的日志记录和更新**

```python
stats.update(timing)
```

- **作用**：更新 **训练过程中的时间统计信息**，并返回当前的训练状态统计信息。


critic model 是在哪生成的？

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204145924432.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204150046339.png" width=100%></div>

**logits**：这是 **主模型（Actor Model）** 生成的动作的对数概率，用于计算 **策略损失**（policy loss）。

**_**：这个值没有被使用，通常是模型的其他输出（如注意力权重等）。

**values**：这是来自 **Value Head Model** 的输出，表示 **状态的价值**，用于计算 **价值损失**（value loss）和 **优势函数**（Advantage）。

# 5. 总结

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204154710840.png" width=100%></div>